In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/My Drive/00Pix2Vox00/

/content/drive/My Drive/00Pix2Vox00


In [ ]:
!ls

2D-views-to-3D-Objects	Pix2Vox-A-ShapeNet.pth	TestsP2V
Pix2Vox			Shapenet2dto3d


In [ ]:
cd 2D-views-to-3D-Objects/

/content/drive/My Drive/00Pix2Vox00/2D-views-to-3D-Objects


In [ ]:
!ls

 3D_p0_Pro3v.png       Dockerfile	    __pycache__
 3D_p1_Pro3v.png       environments	    raw.githubusercontent.com
 3D_reco_Pieza1.png    images_passed	    README.md
 3D_reco_Pieza2.png    inference_trial.py   requirements.txt
 3D_reco.png	       LargeDatasets	    runner.py
'Add _Padding.ipynb'   LICENSE		    SemanticSegmentation.ipynb
 app.py		       load_images0.jpg     streamlit.py
 config.py	       models		    Untitled.ipynb
 core		       output		    utils
 datasets	       packages.txt
 DemoTest.ipynb        pix3d_mask.ipynb


In [ ]:
!pip install -r requirements.txt

In [ ]:
import logging
import matplotlib
import multiprocessing as mp
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.backends.cudnn
import torch.utils.data
import torchvision.transforms

import utils.binvox_visualization
import utils.data_loaders
import utils.data_transforms
import utils.network_utils

# Fix problem: no $DISPLAY environment variable
matplotlib.use('Agg')

from argparse import ArgumentParser
from datetime import datetime as dt
from pprint import pprint

from config import cfg
from core.train import train_net
#from core.test import test_net
#from core.inference import inference_net
from core.demo import test_net
#from core.demo_trial import test_net
from torch.utils.tensorboard import SummaryWriter
import skimage.measure as sm

In [ ]:
#PATH = '/content/2D-views-to-3D-Objects/Pix2Vox-A-ShapeNet.pth'
PATH = '/content/drive/MyDrive/00Pix2Vox00/Pix2Vox/output/best-ckpt.pth'
torch.backends.cudnn.benchmark = True

#checkpoint = (torch.load(PATH))

print('Use config:')
pprint(cfg)

cfg.CONST.WEIGHTS = '/content/drive/MyDrive/00Pix2Vox00/Pix2Vox/output/best-ckpt.pth'
#cfg.CONST.WEIGHTS = './pretrained_models/ckpt-epoch-0550.pth'

writer = SummaryWriter('./output/tensorboard')
generated_volume, rendering_images = test_net(cfg,output_dir='./output')

print(generated_volume.shape)
volume = generated_volume.reshape(32,32,32)
print(volume.shape)

Use config:
{'CONST': {'BATCH_SIZE': 64,
           'CROP_IMG_H': 128,
           'CROP_IMG_W': 128,
           'DEVICE': '0',
           'IMG_H': 224,
           'IMG_W': 224,
           'N_VIEWS_RENDERING': 3,
           'N_VOX': 32,
           'RNG_SEED': 0},
 'DATASET': {'MEAN': [0.5, 0.5, 0.5],
             'STD': [0.5, 0.5, 0.5],
             'TEST_DATASET': 'Inference'},
 'DATASETS': {'INFERENCE': {'RENDERING_PATH': '/content/drive/MyDrive/00Pix2Vox00/Shapenet2dto3d/datasets/DemoImage/%s/%s/rendering/%02d.png',
                            'TAXONOMY_FILE_PATH': '/content/drive/MyDrive/00Pix2Vox00/Shapenet2dto3d/datasets/DemoDataPieza2.json'}},
 'DIR': {'OUT_PATH': './output',
         'RANDOM_BG_PATH': '/home/hzxie/Datasets/SUN2012/JPEGImages'},
 'NETWORK': {'LEAKY_VALUE': 0.2,
             'TCONV_USE_BIAS': False,
             'USE_MERGER': True,
             'USE_REFINER': True},
 'TEST': {'RANDOM_BG_COLOR_RANGE': [[240, 240], [240, 240], [240, 240]],
          'VOXEL_THRESH': 

2021-07-09 00:21:01.605043: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


[INFO] 2021-07-09 00:21:02.840702 Collecting files of Taxonomy[ID=pieza2, Name=pieza2]
[INFO] 2021-07-09 00:21:02.843141 Complete collecting files of the dataset. Total files: 1.
[INFO] 2021-07-09 00:21:08.466843 Loading weights from /content/drive/MyDrive/00Pix2Vox00/Pix2Vox/output/best-ckpt.pth ...
Epoch ID of the current model is 150
test data loader type is <class 'torch.utils.data.dataloader.DataLoader'>
sample IDx 0
taxonomy id pieza2
Shape of the loaded images torch.Size([1, 3, 3, 224, 224])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


vox shape torch.Size([1, 32, 32, 32])
gv shape is (1, 32, 32, 32)
(1, 32, 32, 32)
(32, 32, 32)


In [ ]:
img_array = rendering_images.cpu().numpy()
print(len(img_array))
print(img_array.shape)
for i in range(0,1):
    image_array = img_array[:,i,:,:,:]
    print(image_array.shape)
    image_array = image_array.reshape(image_array.shape[2],image_array.shape[3],3)
    print(image_array.shape)
    %matplotlib notebook
    fig = plt.figure(i)
    plt.imshow(image_array)
    print("mean = {}".format(np.mean(image_array)))

1
(1, 3, 3, 224, 224)
(1, 3, 224, 224)
(224, 224, 3)


<IPython.core.display.Javascript object>

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


mean = 0.1747559756040573


In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume_plot = volume.squeeze().__ge__(0.3)
ax.voxels(volume_plot, facecolor='k', edgecolor="b")
ax.view_init(0,70)
ax.get_xaxis().set_visible(True)
ax.get_yaxis().set_visible(True)
plt.axis('off')
plt.show()
plt.savefig("3D_p2_Pro3v.png", bbox_inches='tight')

<IPython.core.display.Javascript object>